<a href="https://colab.research.google.com/github/Ricardo50-dev/Graph_generator/blob/main/Graph_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.metrics import pairwise
from sklearn.neighbors import DistanceMetric
import igraph as ig

In [ ]:
def Assortativity(graph):
    return ig.Graph.assortativity_degree(graph)


def ClustCoefficient(graph):
    return ig.Graph.transitivity_avglocal_undirected(graph)


def AvgDegree(graph):
    return np.mean(ig.Graph.degree(graph))


def Betweenness(graph):
    return np.mean(ig.Graph.betweenness(graph))


def AvgPathLength(graph):
    return ig.Graph.average_path_length(graph)


def Closeness(graph):
    return np.mean(ig.Graph.closeness(graph))

In [2]:
def eN_Graph(raio, X, y):

    # pega o num de classes
    numclasses = len(np.unique(y))

    # X = gauss_spline(X, len(X))
    euclidean_dist = pairwise.cosine_similarity(X)
    #dist = DistanceMetric.get_metric('euclidian')
    #euclidean_dist = dist.pairwise(X)


    # preenche a diagonal principal com valor infinito
    np.fill_diagonal(euclidean_dist, np.inf)


    # ordena as distancias por objeto em ordem ascendente
    ind_ranking = np.argsort(euclidean_dist, axis=1)[:, :]
    #print(euclidean_dist)
    # inicializa uma mascara com False, num de objetos x k
    mask = np.zeros((len(X), len(X))).astype(int)

    # preenche os campos True da mascara, vizinho mais proximos que sao da mesma clase
    lista_grafos = []
    for i in range(len(ind_ranking)):
        for j in range(len(ind_ranking)):
            if y[ind_ranking[i][j]] == y[i] and euclidean_dist[i][j] <= raio:
                mask[i][j] = True

    # captura os indices True da mascara, linha e coluna
    links = mask.nonzero()

    # atribui os indices das linhas
    sources = links[0]

    # atribui o indice real dos objetos das colunas
    targets = ind_ranking[links]

    # inicializa funcao para mapeamento dos vertices
    map_vertices = np.zeros(len(ind_ranking)).astype(int) - 1

    # inicializa variavel para guardar o calculo das medidas
    measures = np.zeros((numclasses, 6))
    for l in range(numclasses):

        # captura todos os objetos em sources que pertencem a classe l
        lsources = np.where(y[sources] == l)[0]

        # captura todos os objetos da base que pertencem a classe l
        all_vertices = np.where(y == l)[0]

        # recebe todos os objetos da classe l que estao conectados
        unique_vertices = np.unique(
            np.append(sources[lsources], targets[lsources])).astype(int)

        # recebe os demais objetos da classe l que nao estao conectados
        unique_vertices = np.unique(
            np.append(unique_vertices, all_vertices)).astype(int)

        # popula a funcao de mapeamento dos vertices com os objetos da classe l
        map_vertices[unique_vertices] = np.arange(
            len(unique_vertices)).astype(int)

        # print(len(unique_vertices), unique_vertices)

        # cria o grafo para classe l
        subg = ig.Graph(len(unique_vertices), list(zip(map_vertices[sources[lsources]].astype(
            int), map_vertices[targets[lsources]].astype(int))))

        #print(subg.vcount())

        # popula a lista de grafos
        lista_grafos.append(subg)

        # calcula as medidas de rede associadas ao grafo l
        measures[l, 0] = Assortativity(subg)
        measures[l, 1] = ClustCoefficient(subg)
        measures[l, 2] = AvgDegree(subg)
        measures[l, 3] = Betweenness(subg)
        measures[l, 4] = AvgPathLength(subg)
        measures[l, 5] = Closeness(subg)


    return lista_grafos, map_vertices, measures

In [ ]:
def kNN_Graph(k, X, y):

    # pega o num de classes
    numclasses = len(np.unique(y))

    # X = gauss_spline(X, len(X))
    euclidean_dist = pairwise.cosine_similarity(X)
    #dist = DistanceMetric.get_metric('euclidian')
    #euclidean_dist = dist.pairwise(X)


    # preenche a diagonal principal com valor infinito
    np.fill_diagonal(euclidean_dist, np.inf)

    # ordena as distancias por objeto em ordem ascendente
    ind_ranking = np.argsort(euclidean_dist, axis=1)[:, :k]

    # inicializa uma mascara com False, num de objetos x k
    mask = np.zeros((len(X), k)).astype(int)

    # preenche os campos True da mascara, vizinho mais proximos que sao da mesma clase
    lista_grafos = []
    for i in range(len(ind_ranking)):
        mask[i] = (y[ind_ranking[i]] == y[i])

    # captura os indices True da mascara, linha e coluna
    links = mask.nonzero()

    # atribui os indices das linhas
    sources = links[0]

    # atribui o indice real dos objetos das colunas
    targets = ind_ranking[links]

    # inicializa funcao para mapeamento dos vertices
    map_vertices = np.zeros(len(ind_ranking)).astype(int) - 1

    # inicializa variavel para guardar o calculo das medidas
    measures = np.zeros((numclasses, 6))
    for l in range(numclasses):

        # captura todos os objetos em sources que pertencem a classe l
        lsources = np.where(y[sources] == l)[0]

        # captura todos os objetos da base que pertencem a classe l
        all_vertices = np.where(y == l)[0]

        # recebe todos os objetos da classe l que estao conectados
        unique_vertices = np.unique(
            np.append(sources[lsources], targets[lsources])).astype(int)

        # recebe os demais objetos da classe l que nao estao conectados
        unique_vertices = np.unique(
            np.append(unique_vertices, all_vertices)).astype(int)

        # popula a funcao de mapeamento dos vertices com os objetos da classe l
        map_vertices[unique_vertices] = np.arange(
            len(unique_vertices)).astype(int)

        # print(len(unique_vertices), unique_vertices)

        # cria o grafo para classe l
        subg = ig.Graph(len(unique_vertices), list(zip(map_vertices[sources[lsources]].astype(
            int), map_vertices[targets[lsources]].astype(int))))

        #print(subg.vcount())

        # popula a lista de grafos
        lista_grafos.append(subg)

        # calcula as medidas de rede associadas ao grafo l
        measures[l, 0] = Assortativity(subg)
        measures[l, 1] = ClustCoefficient(subg)
        measures[l, 2] = AvgDegree(subg)
        measures[l, 3] = Betweenness(subg)
        measures[l, 4] = AvgPathLength(subg)
        measures[l, 5] = Closeness(subg)


    return lista_grafos, map_vertices, measures

In [ ]:
def S_kNN_Graph(k, X, y):

    numclasses = len(np.unique(y))
    lista_grafos = []
    # inicializa variavel para guardar o calculo das medidas
    measures = np.zeros((numclasses, 6))
    # inicializa funcao para mapeamento dos vertices
    map_vertices = np.zeros(len(X)).astype(int) - 1
    for l in range(0, numclasses):

        classes = X[np.where(y == l)[0]]

        euclidean_dist = pairwise.cosine_similarity(classes)
        #dist = DistanceMetric.get_metric('euclidian')
        #euclidean_dist = dist.pairwise(classes)

        # preenche a diagonal principal com valor infinito
        np.fill_diagonal(euclidean_dist, np.inf)

        # ordena as distancias por objeto em ordem ascendente
        ind_ranking = np.argsort(euclidean_dist, axis=1)[:, :k]

        # inicializa uma mascara com False, num de objetos x k
        mask = np.zeros((len(classes), k)).astype(int)

        # preenche os campos True da mascara, vizinho mais proximos que sao da mesma clase

        for i in range(len(ind_ranking)):
            mask[i] = True

        # captura os indices True da mascara, linha e coluna
        links = mask.nonzero()

        # atribui os indices das linhas
        sources = links[0]

        # atribui o indice real dos objetos das colunas
        targets = ind_ranking[links]

        # captura todos os objetos em sources que pertencem a classe l
        lsources = np.where(y[sources] == l)[0]

        # captura todos os objetos da base que pertencem a classe l
        all_vertices = np.where(y == l)[0]

        # recebe todos os objetos da classe l que estao conectados
        unique_vertices = np.unique(
            np.append(sources[lsources], targets[lsources])).astype(int)

        # recebe os demais objetos da classe l que nao estao conectados
        unique_vertices = np.unique(
            np.append(unique_vertices, all_vertices)).astype(int)

        # popula a funcao de mapeamento dos vertices com os objetos da classe l
        map_vertices[unique_vertices] = np.arange(
            len(unique_vertices)).astype(int)
        # print(len(unique_vertices), unique_vertices)

        # cria o grafo para classe l
        subg = ig.Graph(len(unique_vertices), list(zip(map_vertices[sources[lsources]].astype(
            int), map_vertices[targets[lsources]].astype(int))))

        #print(subg.vcount())

        # popula a lista de grafos
        lista_grafos.append(subg)

        # calcula as medidas de rede associadas ao grafo l
        measures[l, 0] = Assortativity(subg)
        measures[l, 1] = ClustCoefficient(subg)
        measures[l, 2] = AvgDegree(subg)
        measures[l, 3] = Betweenness(subg)
        measures[l, 4] = AvgPathLength(subg)
        measures[l, 5] = Closeness(subg)

    return lista_grafos, map_vertices, measures

In [ ]:
def M_kNN_Graph(k, X, y):

    numclasses = len(np.unique(y))
    lista_grafos = []
    # inicializa variavel para guardar o calculo das medidas
    measures = np.zeros((numclasses, 6))
    # inicializa funcao para mapeamento dos vertices
    map_vertices = np.zeros(len(X)).astype(int) - 1
    for l in range(0, numclasses):

        classes = X[np.where(y == l)[0]]

        euclidean_dist = pairwise.cosine_similarity(classes)
        #dist = DistanceMetric.get_metric('euclidian')
        #euclidean_dist = dist.pairwise(classes)

        # preenche a diagonal principal com valor infinito
        np.fill_diagonal(euclidean_dist, np.inf)

        # ordena as distancias por objeto em ordem ascendente
        ind_ranking = np.argsort(euclidean_dist, axis=1)[:, :k]
        # inicializa uma mascara com False, num de objetos x k
        mask = np.zeros((len(classes), k)).astype(int)

        # preenche os campos True da mascara, vizinho mais proximos que sao da mesma clase

        for i in range(len(ind_ranking)):
          for j in range(0, k):
            if ind_ranking[i][j] in ind_ranking[ind_ranking[i][j] - 1]:
                mask[i][j] = True

            else:
                mask[i][j] = False

        # captura os indices True da mascara, linha e coluna
        links = mask.nonzero()

        # atribui os indices das linhas
        sources = links[0]

        # atribui o indice real dos objetos das colunas
        targets = ind_ranking[links]

        # captura todos os objetos em sources que pertencem a classe l
        lsources = np.where(y[sources] == l)[0]

        # captura todos os objetos da base que pertencem a classe l
        all_vertices = np.where(y == l)[0]

        # recebe todos os objetos da classe l que estao conectados
        unique_vertices = np.unique(
            np.append(sources[lsources], targets[lsources])).astype(int)

        # recebe os demais objetos da classe l que nao estao conectados
        unique_vertices = np.unique(
            np.append(unique_vertices, all_vertices)).astype(int)

        # popula a funcao de mapeamento dos vertices com os objetos da classe l
        map_vertices[unique_vertices] = np.arange(
            len(unique_vertices)).astype(int)
        # print(len(unique_vertices), unique_vertices)

        # cria o grafo para classe l
        subg = ig.Graph(len(unique_vertices), list(zip(map_vertices[sources[lsources]].astype(
            int), map_vertices[targets[lsources]].astype(int))))

        #print(subg.vcount())

        # popula a lista de grafos
        lista_grafos.append(subg)

        # calcula as medidas de rede associadas ao grafo l
        measures[l, 0] = Assortativity(subg)
        measures[l, 1] = ClustCoefficient(subg)
        measures[l, 2] = AvgDegree(subg)
        measures[l, 3] = Betweenness(subg)
        measures[l, 4] = AvgPathLength(subg)
        measures[l, 5] = Closeness(subg)

    return lista_grafos, map_vertices, measures